# Project 1: Exploratory Data Analysis

## MTA Turnstile Dataset

### Chris Doenlen, Vanessa Hu, Jay Park, Matt Ranalletta

#### Sources & Reference
- [MTA Turnstile Data](http://web.mta.info/developers/turnstile.html)
- [MTA Turnstile Data - Codebook](http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt)
- [MTA NYC Subway Map](http://web.mta.info/maps/Large_Print_Map.pdf)
- [Kaggle: MTA Turnstile Data Analysis](https://www.kaggle.com/nieyuqi/mta-turnstile-data-analysis)

### Data Compilation and Cleaning

1. Retrieve 6 weeks of MTA Turnstile data (weeks starting Aug 10, 2019 through Sept 14, 2019) and compile into a single dataframe
2. Clean raw data and perform basic manipulations and calculations
3. Export final dataset to csv to be used in subsequent notebooks

#### Raw Data Import and Treatment

In [1]:
import numpy as np
import pandas as pd

import datetime

In [2]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

In [3]:
import_weeks = [190914, 190907, 190831, 190824, 190817, 190810]

mta_raw = get_data(import_weeks)

In [4]:
mta_raw.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,00:00:00,REGULAR,7190495,2435566
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,04:00:00,REGULAR,7190516,2435574
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,08:00:00,REGULAR,7190538,2435603
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,12:00:00,REGULAR,7190676,2435676
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/07/2019,16:00:00,REGULAR,7190931,2435725


In [5]:
# Rename columns for easier coding later

mta_raw.columns = ['ca',
                   'unit',
                  'scp',
                  'station',
                  'linename',
                  'division',
                  'date',
                  'time',
                  'desc',
                  'entries_cum',
                  'exits_cum']

In [6]:
# Create a timestamp column with datetime object
# Convert date data to datetime object

mta_raw['timestamp'] = pd.to_datetime(mta_raw['date'] + ' ' + mta_raw['time'])
mta_raw['date'] = pd.to_datetime(mta_raw['date'])

In [7]:
# Create turnstile column as proxy for unique identifier
mta_raw['turnstile'] = mta_raw['station'] + '-' + mta_raw['ca'] + '-' + mta_raw['unit'] + '-' + mta_raw['scp']

#### Searching for duplicates

In [8]:
dupes = mta_raw.groupby(['turnstile', 'timestamp'])['entries_cum'].count().reset_index().sort_values('entries_cum', ascending=False).head(15)

In [9]:
dupes

,turnstile,timestamp,entries_cum
392165,72 ST-R161A-R452-01-00-02,2019-08-26 21:00:00,2
61942,14 ST-UNION SQ-A034-R170-03-05-03,2019-08-11 17:00:00,2
760356,FRANKLIN AV-R622-R123-00-00-06,2019-08-22 04:59:02,2
1005878,NEW LOTS-H039-R375-00-00-01,2019-09-13 05:00:00,2
61952,14 ST-UNION SQ-A034-R170-03-05-03,2019-08-13 09:00:00,2
61951,14 ST-UNION SQ-A034-R170-03-05-03,2019-08-13 05:00:00,2
760613,FRANKLIN AV-R622-R123-00-00-07,2019-08-22 04:59:02,2
61950,14 ST-UNION SQ-A034-R170-03-05-03,2019-08-13 01:00:00,2
61949,14 ST-UNION SQ-A034-R170-03-05-03,2019-08-12 21:00:00,2
61948,14 ST-UNION SQ-A034-R170-03-05-03,2019-08-12 17:00:00,2


It seems like one aprticualr turnstile -- `14 ST-UNION SQ-A034-R170-03-05-03` -- makes up the majority of the duplicates in the list.

Digging into a particular date -- see below -- it looks like for this particular turnstile at `14 ST-UNION SQ` station, every interval has both a `REGULAR` and a `RECOVR AUD` entry. For each entry type, the delta between a current and previous period is about the same. However, the difference between types can be over 1,000. If we were to calculate actual entries/exits based on the difference between two rows, the different entry types could seriously skew our data. 

Thus, we should drop all `RECOVR AUD` entries and keep only `REGULAR` for this particular turnstile.

In [10]:
mta_raw[(mta_raw['turnstile']=='14 ST-UNION SQ-A034-R170-03-05-03') & 
       (mta_raw['date']=='2019-08-11')]

,ca,unit,scp,station,linename,division,date,time,desc,entries_cum,exits_cum,timestamp,turnstile
5236,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,01:00:00,REGULAR,26595,21402,2019-08-11 01:00:00,14 ST-UNION SQ-A034-R170-03-05-03
5237,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,01:00:00,RECOVR AUD,25723,30974,2019-08-11 01:00:00,14 ST-UNION SQ-A034-R170-03-05-03
5238,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,05:00:00,REGULAR,26619,21423,2019-08-11 05:00:00,14 ST-UNION SQ-A034-R170-03-05-03
5239,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,05:00:00,RECOVR AUD,25742,30989,2019-08-11 05:00:00,14 ST-UNION SQ-A034-R170-03-05-03
5240,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,09:00:00,REGULAR,26639,21497,2019-08-11 09:00:00,14 ST-UNION SQ-A034-R170-03-05-03
5241,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,09:00:00,RECOVR AUD,25752,31054,2019-08-11 09:00:00,14 ST-UNION SQ-A034-R170-03-05-03
5242,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,13:00:00,REGULAR,26747,21775,2019-08-11 13:00:00,14 ST-UNION SQ-A034-R170-03-05-03
5243,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,13:00:00,RECOVR AUD,25845,31226,2019-08-11 13:00:00,14 ST-UNION SQ-A034-R170-03-05-03
5244,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,17:00:00,REGULAR,26948,22121,2019-08-11 17:00:00,14 ST-UNION SQ-A034-R170-03-05-03
5245,A034,R170,03-05-03,14 ST-UNION SQ,LNQR456W,BMT,2019-08-11,17:00:00,RECOVR AUD,26029,31456,2019-08-11 17:00:00,14 ST-UNION SQ-A034-R170-03-05-03


Checking another duplicate -- seems like the delta between the `REGULAR` and `RECOVR AUD` entries is just the counter starting over, so the value will be high enough that we can ignore it later as an outlier. 

In [11]:
mta_raw[(mta_raw['turnstile']=='72 ST-R161A-R452-01-00-02') & 
       (mta_raw['date']=='2019-08-26')]

,ca,unit,scp,station,linename,division,date,time,desc,entries_cum,exits_cum,timestamp,turnstile
142242,R161A,R452,01-00-02,72 ST,123,IRT,2019-08-26,01:00:00,REGULAR,2543403,2592663,2019-08-26 01:00:00,72 ST-R161A-R452-01-00-02
142243,R161A,R452,01-00-02,72 ST,123,IRT,2019-08-26,05:00:00,REGULAR,2543406,2592671,2019-08-26 05:00:00,72 ST-R161A-R452-01-00-02
142244,R161A,R452,01-00-02,72 ST,123,IRT,2019-08-26,09:00:00,REGULAR,2544254,2592765,2019-08-26 09:00:00,72 ST-R161A-R452-01-00-02
142245,R161A,R452,01-00-02,72 ST,123,IRT,2019-08-26,13:00:00,REGULAR,2545255,2593027,2019-08-26 13:00:00,72 ST-R161A-R452-01-00-02
142246,R161A,R452,01-00-02,72 ST,123,IRT,2019-08-26,17:00:00,RECOVR AUD,2547117,2593174,2019-08-26 17:00:00,72 ST-R161A-R452-01-00-02
142247,R161A,R452,01-00-02,72 ST,123,IRT,2019-08-26,21:00:00,REGULAR,2548234,2593726,2019-08-26 21:00:00,72 ST-R161A-R452-01-00-02
142248,R161A,R452,01-00-02,72 ST,123,IRT,2019-08-26,21:00:00,RECOVR AUD,1,0,2019-08-26 21:00:00,72 ST-R161A-R452-01-00-02


Drop all rows for turnstile `14 ST-UNION SQ-A034-R170-03-05-03` that are `RECOVR AUD`: 

In [12]:
trouble_turnstile_index = mta_raw[(mta_raw['turnstile']=='14 ST-UNION SQ-A034-R170-03-05-03') & 
                   (mta_raw['desc']=='RECOVR AUD')].index

mta_raw.drop(trouble_turnstile_index, inplace=True)

#### Calculating actual entries and exits from cumulative figures

Using `.transform(pd.Series.diff)` as demonstrated on [Kaggle's MTA Turnstile Data Analysis](https://www.kaggle.com/nieyuqi/mta-turnstile-data-analysis) and applying `.abs()` to account for negative entries from potential backwards counting turnstiles.

In [13]:
mta_sorted = mta_raw.sort_values(['turnstile', 'timestamp'])
mta_sorted = mta_sorted.reset_index(drop = True)

turnstile_grouped = mta_sorted.groupby(['turnstile'])

mta_sorted['entries'] = turnstile_grouped['entries_cum'].transform(pd.Series.diff).abs()
mta_sorted['exits'] = turnstile_grouped['exits_cum'].transform(pd.Series.diff).abs()

In [14]:
mta_sorted.head()

,ca,unit,scp,station,linename,division,date,time,desc,entries_cum,exits_cum,timestamp,turnstile,entries,exits
0,H007,R248,00-00-00,1 AV,L,BMT,2019-08-03,00:00:00,REGULAR,14885151,16634862,2019-08-03 00:00:00,1 AV-H007-R248-00-00-00,NaN,NaN
1,H007,R248,00-00-00,1 AV,L,BMT,2019-08-03,04:00:00,REGULAR,14885151,16634872,2019-08-03 04:00:00,1 AV-H007-R248-00-00-00,0.0,10.0
2,H007,R248,00-00-00,1 AV,L,BMT,2019-08-03,08:00:00,REGULAR,14885151,16634879,2019-08-03 08:00:00,1 AV-H007-R248-00-00-00,0.0,7.0
3,H007,R248,00-00-00,1 AV,L,BMT,2019-08-03,12:00:00,REGULAR,14885151,16634889,2019-08-03 12:00:00,1 AV-H007-R248-00-00-00,0.0,10.0
4,H007,R248,00-00-00,1 AV,L,BMT,2019-08-03,16:00:00,REGULAR,14885151,16634901,2019-08-03 16:00:00,1 AV-H007-R248-00-00-00,0.0,12.0


In [15]:
# Delete mta_raw
del mta_raw

#### Dealing with outliers and messy data

Two messy data type: 
* **Very large values**: some turnstile counts are unbelievably large. We'll set a threshold of 10,000 entries or exits per turnstile per time period (this translates to ~40 entries/exits per minute, which is feasible). Any values above this threshold will be converted to NaN and dropped.
* **Not a number (NaN)**: the majority of these NaN values is because they were the start of our timeperiod for the turnstile and thus had no prior time period to calculate the actual values from the cumulative figures. 

A third, **negative values** would have been handled above when we applied the `abs()` method above.

These cases will be converted to NaN (if not already NaN) and dropped from the dataset. 

In [16]:
print('Number of negative entries: %d' %len(mta_sorted['entries'][mta_sorted['entries'] < 0]))
print('Number of negative exits: %d' %len(mta_sorted['exits'][mta_sorted['exits'] < 0]))
print('')
print('Number of entries > 10k: %d' %len(mta_sorted['entries'][mta_sorted['entries'] > 10000]))
print('Number of exits > 10k: %d' %len(mta_sorted['exits'][mta_sorted['exits'] > 10000]))
print('')
print('Number of NaN rows: %d' %len(mta_sorted[mta_sorted['entries'].isnull()]))

Number of negative entries: 0
Number of negative exits: 0

Number of entries > 10k: 141
Number of exits > 10k: 121

Number of NaN rows: 4934


In [17]:
# Filtering for values above our 10k threshold

ents_10k = mta_sorted.loc[:, 'entries'] > 10000
exits_10k = mta_sorted.loc[:, 'exits'] > 10000

In [18]:
# Converting negative and above threshold entries to Nan

mta_sorted.loc[ents_10k, 'entries'] = np.nan
mta_sorted.loc[exits_10k, 'exits'] = np.nan

In [19]:
print('Number of negative entries: %d' %len(mta_sorted['entries'][mta_sorted['entries'] < 0]))
print('Number of negative exits: %d' %len(mta_sorted['exits'][mta_sorted['exits'] < 0]))
print('')
print('Number of entries > 10k: %d' %len(mta_sorted['entries'][mta_sorted['entries'] > 10000]))
print('Number of exits > 10k: %d' %len(mta_sorted['exits'][mta_sorted['exits'] > 10000]))
print('')
print('Number of NaN rows: %d' %len(mta_sorted[mta_sorted['entries'].isnull()]))

Number of negative entries: 0
Number of negative exits: 0

Number of entries > 10k: 0
Number of exits > 10k: 0

Number of NaN rows: 5075


In [20]:
# Dropping na values

mta_sorted.dropna(inplace=True)

In [21]:
print('Number of NaN rows: %d' %len(mta_sorted[mta_sorted['entries'].isnull()]))

Number of NaN rows: 0


#### Total activity per turnstile

Creating a new row called `total` that will be the total volume through a turnstile (`entries` + `exits`)

In [22]:
mta_sorted['total'] = mta_sorted['entries'] + mta_sorted['exits']

#### Number of lines at a station

In anticipation of needing to better identify stations best suited to serve the client's need, we'll add a column counting the number of lines passing through a particular location -- this will help us target more active stations.

In [23]:
mta_sorted['num_lines'] = mta_sorted['linename'].str.len()

#### Day of the week

Adding a column called `weekday` that gives us the day of the week for easier analysis later. 

In [25]:
mta_sorted['weekday'] = mta_sorted['date'].dt.day_name()

In [27]:
mta_sorted.head(3)

,ca,unit,scp,station,linename,division,date,time,desc,entries_cum,exits_cum,timestamp,turnstile,entries,exits,total,num_lines,weekday
1,H007,R248,00-00-00,1 AV,L,BMT,2019-08-03,04:00:00,REGULAR,14885151,16634872,2019-08-03 04:00:00,1 AV-H007-R248-00-00-00,0.0,10.0,10.0,1,Saturday
2,H007,R248,00-00-00,1 AV,L,BMT,2019-08-03,08:00:00,REGULAR,14885151,16634879,2019-08-03 08:00:00,1 AV-H007-R248-00-00-00,0.0,7.0,7.0,1,Saturday
3,H007,R248,00-00-00,1 AV,L,BMT,2019-08-03,12:00:00,REGULAR,14885151,16634889,2019-08-03 12:00:00,1 AV-H007-R248-00-00-00,0.0,10.0,10.0,1,Saturday


#### Creating clean, organized dataframe to be exported as csv and used for analysis

In [28]:
mta = mta_sorted[['station', 
                'turnstile',
                 'ca',
                 'unit',
                 'scp',
                 'linename',
                 'num_lines',
                 'division',
                 'desc',
                 'date',
                 'time',
                 'timestamp',
                 'weekday',
                 'entries',
                 'exits',
                 'total']]

In [29]:
mta.head(3)

,station,turnstile,ca,unit,scp,linename,num_lines,division,desc,date,time,timestamp,weekday,entries,exits,total
1,1 AV,1 AV-H007-R248-00-00-00,H007,R248,00-00-00,L,1,BMT,REGULAR,2019-08-03,04:00:00,2019-08-03 04:00:00,Saturday,0.0,10.0,10.0
2,1 AV,1 AV-H007-R248-00-00-00,H007,R248,00-00-00,L,1,BMT,REGULAR,2019-08-03,08:00:00,2019-08-03 08:00:00,Saturday,0.0,7.0,7.0
3,1 AV,1 AV-H007-R248-00-00-00,H007,R248,00-00-00,L,1,BMT,REGULAR,2019-08-03,12:00:00,2019-08-03 12:00:00,Saturday,0.0,10.0,10.0


In [30]:
del mta_sorted

### Export clean dataframe to CSV for later use

*NOTE: Saved as zip file for GitHub limits

In [31]:
mta.to_csv('mta_clean.zip', index=False)

Now that we have a clean dataset, we can get started with exploratory data analysis!